In [7]:
#Importing libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import gc
import PIL.Image as Image
import tifffile as tiff
from pathlib import Path
from tqdm.notebook import tqdm 

#Importing ML Tools
import tensorflow as tf
from tensorflow import keras
print("Tensorflow Version Installed: " + tf.__version__)

physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

!nvidia-smi

print("\nSetup Complete")

Tensorflow Version Installed: 2.10.0
Wed Apr  5 12:54:07 2023       
Setup Complete

+-----------------------------------------------------------------------------+
| NVIDIA-SMI 527.27       Driver Version: 527.27       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 3000    WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   61C    P8     7W /  N/A |      0MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                   

In [11]:
OUTPUT_PNG = True
OUTPUT_NPY = False

ROI_SIZE = 1024
IMAGE_NUM = 3

INPUT_DIR = Path("kaggle/input/vesuvius-challenge-ink-detection/train")
OUTPUT_DIR = Path("kaggle/working")

In [12]:
def crop_image(image_id, input_dir, output_dir):

    output_data_dir = Path(output_dir / f"{image_id}")
    if not output_data_dir.exists():
        output_data_dir.mkdir()
    
    #Load Label image
    inklabels_img = np.array(Image.open(str(input_dir / f"{image_id}_inklabels.png")))
    inklabels_img = np.pad(inklabels_img, [(0, ROI_SIZE-inklabels_img.shape[0]%ROI_SIZE), (0, ROI_SIZE-inklabels_img.shape[1]%ROI_SIZE)], mode='constant')

    #Get the scanning position of the Image
    x_post_list = []
    y_post_list = []
    for y in range(0, inklabels_img.shape[0], ROI_SIZE):
        for x in range(0, inklabels_img.shape[1], ROI_SIZE):
            if inklabels_img[y:y+ROI_SIZE, x:x+ROI_SIZE].max() > 0:
                x_post_list.append(x)
                y_post_list.append(y)
    
    #Crop the image
    image_path_list = sorted(list(Path(input_dir / f"{image_id}" / "surface_volume").glob('*.tif')))
    for i, image_path in tqdm(enumerate(image_path_list), total=len(image_path_list), desc=f"Cropping images - {image_id}", dynamic_ncols=True):

        # load image
        img = tiff.imread(str(image_path))
        img = np.pad(img, [(0, ROI_SIZE-img.shape[0]%ROI_SIZE), (0, ROI_SIZE-img.shape[1]%ROI_SIZE)], 'constant')

        # crop
        for j, (x, y) in enumerate(zip(x_pos_list, y_pos_list)):

            image_roi = img[y:y+ROI_SIZE, x:x+ROI_SIZE]
            image_roi = image_roi.astype(np.float32) / 65535.0

            if OUTPUT_NPY:
                np.save(str(output_data_dir / f"{j:03d}_{i:02d}"), image_roi)

            if OUTPUT_PNG:
                output_image_dir = Path(output_data_dir / f"{j:03d}")
                if not output_image_dir.exists():
                    output_image_dir.mkdir()
                cv2.imwrite(str(output_image_dir / f"{i:02d}.png"), (image_roi * 255).astype(np.uint8))

        del img
        gc.collect()

    # Create training data
    image_id_list = []
    roi_id_list = []
    for j, (x, y) in tqdm(enumerate(zip(x_pos_list, y_pos_list)), total=len(x_pos_list), desc=f"Merge images - {image_id}", dynamic_ncols=True):

        # input
        if OUTPUT_NPY:
            image_tiles = []
            for i in range(len(image_path_list)):
                filename = str(output_data_dir / f"{j:03d}_{i:02d}.npy")
                image_tiles.append(np.load(filename))
                os.remove(filename)
            np.save(str(output_data_dir / f"{j:03d}"), np.stack(image_tiles, axis=0))

        # mask
        image_roi = inklabels_img[y:y+ROI_SIZE, x:x+ROI_SIZE]
        cv2.imwrite(str(output_data_dir / f"{j:03d}.png"), (image_roi * 255).astype(np.uint8))

        # metadata
        image_id_list.append(image_id)
        roi_id_list.append(f"{j:03d}")

    # Drawing to check the cropped position
    inklabels_img = (inklabels_img * 255).astype(np.uint8)
    inklabels_img = cv2.cvtColor(inklabels_img, cv2.COLOR_GRAY2BGR)
    for x, y in zip(x_pos_list, y_pos_list):
        inklabels_img = cv2.rectangle(
            inklabels_img,
            (x, y),
            (x+ROI_SIZE, y+ROI_SIZE),
            (0, 255, 0),
            thickness=10)

    cv2.imwrite(str(output_dir / f"crop_image{image_id}.png"), inklabels_img)
    plt.imshow(cv2.cvtColor(inklabels_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

    return {
        "image_id_list": image_id_list,
        "roi_id_list": roi_id_list,
        "x_pos_list": x_pos_list,
        "y_pos_list": y_pos_list,
    }


In [13]:
image_id_list_all = []
roi_id_list_all = []
x_pos_list_all = []
y_pos_list_all = []
for image_id in range(1, IMAGE_NUM+1):

    outputs = crop_image(image_id, INPUT_DIR, OUTPUT_DIR)

    image_id_list_all.extend(outputs["image_id_list"])
    roi_id_list_all.extend(outputs["roi_id_list"])
    x_pos_list_all.extend(outputs["x_pos_list"])
    y_pos_list_all.extend(outputs["y_pos_list"])

df = pd.DataFrame({
    "image_id": image_id_list_all,
    "roi_id": roi_id_list_all,
    "x_pos": x_pos_list_all,
    "y_pos": y_pos_list_all,
})
df.to_csv(str(OUTPUT_DIR / "train.csv"), index=False)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'kaggle\\working\\1'